# 总述
LangChain’s create_agent runs on LangGraph’s runtime under the hood.        
LangGraph exposes a Runtime object with the following information:      
1. **Context**: static information like user id, db connections, or other dependencies for an agent invocation
2. **Store**: a [BaseStore](https://reference.langchain.com/python/langgraph/store/#langgraph.store.base.BaseStore) instance used for [long-term memory](/oss/python/langchain/long-term-memory)
3. **Stream writer**: an object used for streaming information via the `"custom"` stream mode

Access the runtime information within tools and middleware.

*以上这段完全来自官方文档，已经很简单了。但是我不做出点总结，显得我没思考。      
LangChain creat_agent 的底层运行在 LangGraph runtime 中，runtime提供了三类信息：情景、（记忆、状态）存储、自定义流。       
可以用 tool 或 中间件 来访问运行时信息      
也就只是翻译了一下，唉*😔

## Access
````py
    @dataclass
    class Context:
        user_name: str

    agent = create_agent(
        model="gpt-5-nano",
        tools=[...],
        context_schema=Context  
    )

    agent.invoke(
        {"messages": [{"role": "user", "content": "What's my name?"}]},
        context=Context(user_name="John Smith")  
    )
````

### 工具中获取:ToolRuntime
也可以参考我之前的文档：langchain/Core components/Tools.ipynb
````py
    from dataclasses import dataclass
    from langchain.tools import tool, ToolRuntime  

    @dataclass
    class Context:
        user_id: str

    @tool
    def fetch_user_email_preferences(runtime: ToolRuntime[Context]) -> str:  
        """Fetch the user's email preferences from the store."""
        user_id = runtime.context.user_id  

        preferences: str = "The user prefers you to write a brief and polite email."
        if runtime.store:  
            if memory := runtime.store.get(("users",), user_id):  
                preferences = memory.value["preferences"]

        return preferences
````

### 中间件获取
参看[官方文档](https://docs.langchain.com/oss/python/langchain/runtime#inside-middleware)       
大概思路：create dynamic prompts, modify messages, or control agent behavior based on user context.     
因此 大概 涉及 三个 中间件:
````py
    from langchain.agents.middleware import dynamic_prompt, ModelRequest, before_model, after_model
````